In [1]:
import sys
import json
import numpy as np
from cassandra.cluster import Cluster

## Conexión cassandra
``` python
from cassandra.cluster import Cluster

with Cluster() as cluster:
    with cluster.connect('apolo') as session:
        rows = session.execute('SELECT * FROM user')
        stmt = session.prepare("""
                   INSERT INTO activate (rowkey, qualifier, info, act_date, log_time)
                   VALUES (?, ?, ?, ?, ?)
                   IF NOT EXISTS
                   """)
        results = session.execute(stmt, [arg1, arg2, ...])  
```


In [7]:
def insertRestaurantLine(line,session):
    restaurant = json.loads(line)
    stmt = session.prepare('INSERT INTO restaurants (id, location, name, type) values(?, ?, ?, ?)')
    args = [restaurant["_id"]["$oid"], restaurant["location"]["coordinates"], restaurant["name"], restaurant["location"]["type"]]
    session.execute(stmt, args)

In [8]:
with Cluster() as cluster:
    with cluster.connect('challenge') as session:
        with open("./restaurants.json","r") as file:
            while True:
                line = file.readline()
                if line =='':
                    break
                insertRestaurantLine(line, session)
print('Load succesful')

Load succesful


In [10]:
def insertPoligon(obj,session):
    i = 0
    for point in obj["geometry"]["coordinates"][0]:
        stmt = session.prepare('INSERT INTO neighborhoods (id, "polygonNumber", "pointNumber",name, type, point) \
                               values(?, ?, ?, ?, ?, ?)')
        args = [obj["_id"]["$oid"], 0, i, obj['name'], obj["geometry"]["type"], obj["geometry"]["coordinates"][0][i]]
        session.execute(stmt, args) 
        i+=1
        
def insertMultiPoligon(obj,session):
    i = 0
    for _ in obj["geometry"]["coordinates"]:
        j = 0
        for _ in obj["geometry"]["coordinates"][i][0]:

            stmt = session.prepare('INSERT INTO neighborhoods (id, "polygonNumber", "pointNumber",name, type, point) \
                                   values(?, ?, ?, ?, ?, ?)')
            args = [obj["_id"]["$oid"], i, j, obj['name'], obj["geometry"]["type"], obj["geometry"]["coordinates"][i][0][j]]
            session.execute(stmt, args) 
            j+=1  
        i+=1

        
def insertNeighborhood(line, session):
    obj = json.loads(line)
    if obj['geometry']['type'] == 'MultiPolygon':
        insertMultiPoligon(obj,session)
    else:
        insertPoligon(obj,session)
        

In [14]:
%%time
with Cluster() as cluster:
    with cluster.connect('challenge') as session:
        with open("./neighborhoods.json","r") as file:
            i = 0
            while True:
                    line = file.readline()
                    if line =='':
                        break
                    insertNeighborhood(line, session)
                    print(i, 'confirmed')
                    i+=1
        

0 confirmed
1 confirmed
2 confirmed
3 confirmed
4 confirmed
5 confirmed
6 confirmed
7 confirmed
8 confirmed
9 confirmed
10 confirmed
11 confirmed
12 confirmed
13 confirmed
14 confirmed
15 confirmed
16 confirmed
17 confirmed
18 confirmed
19 confirmed
20 confirmed
21 confirmed
22 confirmed
23 confirmed
24 confirmed
25 confirmed
26 confirmed
27 confirmed
28 confirmed
29 confirmed
30 confirmed
31 confirmed
32 confirmed
33 confirmed
34 confirmed
35 confirmed
36 confirmed
37 confirmed
38 confirmed
39 confirmed
40 confirmed
41 confirmed
42 confirmed
43 confirmed
44 confirmed
45 confirmed
46 confirmed
47 confirmed
48 confirmed
49 confirmed
50 confirmed
51 confirmed
52 confirmed
53 confirmed
54 confirmed
55 confirmed
56 confirmed
57 confirmed
58 confirmed
59 confirmed
60 confirmed
61 confirmed
62 confirmed
63 confirmed
64 confirmed
65 confirmed
66 confirmed
67 confirmed
68 confirmed
69 confirmed
70 confirmed
71 confirmed
72 confirmed
73 confirmed
74 confirmed
75 confirmed
76 confirmed
77 confir

In [20]:
with open("./neighborhoods.json","r") as file:
    type1 = ''
    while type1 != 'MultiPolygon':
        obj = json.loads(file.readline())
        type1 = obj["geometry"]["type"]